In [2]:
import pandas as pd

# Load datasets
patient_df = pd.read_csv("C:\\data science\\assignment\\Patient_Data.csv")
billing_df = pd.read_csv("C:\\data science\\assignment\\Billing_Data.csv")

patient_df.info()
billing_df.info()

billing_cols = ['PatientID', 'Department', 'Doctor', 'BillAmount']
patient_billing = patient_df[billing_cols]

patient_cleaned = patient_df.drop(columns=['ReceptionistID', 'CheckInTime'])
print(patient_cleaned )

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   PatientID       6 non-null      int64  
 1   Name            6 non-null      object 
 2   Department      6 non-null      object 
 3   Doctor          6 non-null      object 
 4   BillAmount      4 non-null      float64
 5   ReceptionistID  6 non-null      int64  
 6   CheckInTime     6 non-null      object 
dtypes: float64(1), int64(2), object(4)
memory usage: 468.0+ bytes
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   PatientID         5 non-null      int64
 1   InsuranceCovered  5 non-null      int64
 2   FinalAmount       5 non-null      int64
dtypes: int64(3)
memory usage: 252.0 bytes
   PatientID     Name   Department     Doctor  BillAmount
0 

In [3]:
dept_bill = patient_cleaned.groupby('Department')['BillAmount'].sum()
print(dept_bill)
#Remove duplicate patient records
patient_unique = patient_cleaned.drop_duplicates(subset='PatientID').copy()
print(patient_unique)
#Fill missing BillAmount with mean
mean_bill = patient_unique['BillAmount'].mean()

patient_unique.loc[:, 'BillAmount'] = patient_unique['BillAmount'].fillna(mean_bill)
print(patient_unique.loc[:, 'BillAmount'])



Department
Cardiology     16200.0
Dermatology        0.0
Neurology          0.0
Orthopedics     7500.0
Name: BillAmount, dtype: float64
   PatientID     Name   Department     Doctor  BillAmount
0        101    Alice   Cardiology  Dr. Smith      5000.0
1        102      Bob    Neurology   Dr. John         NaN
2        103  Charlie  Orthopedics    Dr. Lee      7500.0
3        104    David   Cardiology  Dr. Smith      6200.0
4        105      Eva  Dermatology   Dr. Rose         NaN
0    5000.000000
1    6233.333333
2    7500.000000
3    6200.000000
4    6233.333333
Name: BillAmount, dtype: float64


In [4]:
#Merge patient & billing dataset
merged_df = pd.merge(patient_unique, billing_df, on='PatientID',how='inner')
print(merged_df)

   PatientID     Name   Department     Doctor   BillAmount  InsuranceCovered  \
0        101    Alice   Cardiology  Dr. Smith  5000.000000              2000   
1        102      Bob    Neurology   Dr. John  6233.333333              1500   
2        103  Charlie  Orthopedics    Dr. Lee  7500.000000              2500   
3        104    David   Cardiology  Dr. Smith  6200.000000              3000   
4        105      Eva  Dermatology   Dr. Rose  6233.333333              1000   

   FinalAmount  
0         3000  
1         3500  
2         5000  
3         3200  
4         4000  


In [5]:
#Concatenate new patients (row-wise)
new_patients = pd.DataFrame({
    'PatientID': [201, 202],
    'Department': ['ENT', 'Cardiology'],
    'Doctor': ['Dr. Mehta', 'Dr. Rao'],
    'BillAmount': [4500, 12000]
})

final_df = pd.concat([merged_df, new_patients], axis=0, ignore_index=True)
print(final_df)

   PatientID     Name   Department     Doctor    BillAmount  InsuranceCovered  \
0        101    Alice   Cardiology  Dr. Smith   5000.000000            2000.0   
1        102      Bob    Neurology   Dr. John   6233.333333            1500.0   
2        103  Charlie  Orthopedics    Dr. Lee   7500.000000            2500.0   
3        104    David   Cardiology  Dr. Smith   6200.000000            3000.0   
4        105      Eva  Dermatology   Dr. Rose   6233.333333            1000.0   
5        201      NaN          ENT  Dr. Mehta   4500.000000               NaN   
6        202      NaN   Cardiology    Dr. Rao  12000.000000               NaN   

   FinalAmount  
0       3000.0  
1       3500.0  
2       5000.0  
3       3200.0  
4       4000.0  
5          NaN  
6          NaN  


In [6]:
#Concatenate new billing columns (column-wise)
billing_extra = pd.DataFrame({
    'InsuranceCovered': ['Yes'] * len(final_df),
    'FinalAmount': final_df['BillAmount'] * 0.9
})

final_df = pd.concat([final_df, billing_extra], axis=1)
print(final_df)

   PatientID     Name   Department     Doctor    BillAmount InsuranceCovered  \
0        101    Alice   Cardiology  Dr. Smith   5000.000000           2000.0   
1        102      Bob    Neurology   Dr. John   6233.333333           1500.0   
2        103  Charlie  Orthopedics    Dr. Lee   7500.000000           2500.0   
3        104    David   Cardiology  Dr. Smith   6200.000000           3000.0   
4        105      Eva  Dermatology   Dr. Rose   6233.333333           1000.0   
5        201      NaN          ENT  Dr. Mehta   4500.000000              NaN   
6        202      NaN   Cardiology    Dr. Rao  12000.000000              NaN   

   FinalAmount InsuranceCovered  FinalAmount  
0       3000.0              Yes       4500.0  
1       3500.0              Yes       5610.0  
2       5000.0              Yes       6750.0  
3       3200.0              Yes       5580.0  
4       4000.0              Yes       5610.0  
5          NaN              Yes       4050.0  
6          NaN              Ye

In [7]:
#Avoiding duplicate columns
final_df['InsuranceCovered'] = 'Yes'
final_df['FinalAmount'] = final_df['BillAmount'] * 0.9
#handling missing names
final_df['Name'] = final_df['Name'].fillna('New Patient')
print(final_df['Name'])




0          Alice
1            Bob
2        Charlie
3          David
4            Eva
5    New Patient
6    New Patient
Name: Name, dtype: object
